In [ ]:
#pip install azure-cognitiveservices-vision-customvision


In [ ]:
pip install mediapipe opencv-python


In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 서비스 설정
ENDPOINT = ""  # 엔드포인트 URL
PREDICTION_KEY = ""  # Prediction Key
PROJECT_ID = ""  # Custom Vision 프로젝트 ID
PUBLISHED_NAME = "Iteration2"  # 배포된 모델 이름

# API 인증 정보 설정
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, credentials)

def classify_image(file_path):
    """
    이미지 파일을 Custom Vision 모델에 전달하여 분류 결과를 반환합니다.
    :param file_path: 분석할 이미지 파일 경로
    :return: 분류 결과 (태그 이름과 확률)
    """
    with open(file_path, "rb") as image_contents:
        # Custom Vision API 호출
        results = predictor.classify_image(PROJECT_ID, PUBLISHED_NAME, image_contents.read())
    
    # 가장 확률이 높은 예측 결과 반환
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

# 사용 예시
if __name__ == "__main__":
    # 상대 경로 설정
    test_image_path = "473-1-2-21-Z62_A-0000027.jpg"  # 상대 경로
    
    # 이미지 분류 실행
    try:
        result = classify_image(test_image_path)
        # 결과 출력
        print(f"Predicted Tag: {result['tag']}, Probability: {result['probability']:.2f}")
    except FileNotFoundError:
        print(f"Error: Image file '{test_image_path}' not found. Please ensure the file exists in the correct relative path.")
    except Exception as e:
        print(f"Custom Vision API 호출 실패: {e}")


# 애저 + 구글 미디어 파이프


In [ ]:
import cv2
import mediapipe as mp
import math
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 서비스 설정
ENDPOINT = ""
PREDICTION_KEY = ""
PROJECT_ID = ""
PUBLISHED_NAME = "LEG_RAISE"

# API 인증 정보 설정
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, credentials)

def classify_image(file_path):
    """
    이미지 파일을 Custom Vision 모델에 전달하여 분류 결과를 반환합니다.
    :param file_path: 분석할 이미지 파일 경로
    :return: 분류 결과 (태그 이름과 확률)
    """
    with open(file_path, "rb") as image_contents:
        results = predictor.classify_image(PROJECT_ID, PUBLISHED_NAME, image_contents.read())
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

def calculate_angle(a, b, c):
    """
    세 점의 좌표를 사용해 각도를 계산합니다.
    :param a, b, c: 좌표 (x, y)
    :return: 각도 (degree)
    """
    ab = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    magnitude_ab = math.sqrt(ab[0]**2 + ab[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    if magnitude_ab * magnitude_bc == 0:
        return 0
    cos_theta = dot_product / (magnitude_ab * magnitude_bc)
    angle = math.degrees(math.acos(cos_theta))
    return angle

def analyze_pose(image_path):
    """
    이미지에서 자세를 분석하고, 다리의 각도를 계산합니다.
    :param image_path: 분석할 이미지 파일 경로
    :return: 계산된 각도 정보
    """
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found.")
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # 다리의 주요 관절 좌표 추출 (오른쪽 다리 예시)
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # 다리 각도 계산
            angle = calculate_angle(hip, knee, ankle)
            
            # 스켈레톤 그리기
            annotated_image = image.copy()
            mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.imshow('Annotated Image', annotated_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            return angle
        else:
            print("포즈를 감지하지 못했습니다.")
            return None

if __name__ == "__main__":
    test_image_path = "473-1-2-21-Z62_A-0000027.jpg"
    
    try:
        # Custom Vision을 사용한 이미지 분류
        result = classify_image(test_image_path)
        print(f"Predicted Tag: {result['tag']}, Probability: {result['probability']:.2f}")
        
        # 비정상 자세일 경우 스켈레톤 분석 및 피드백
        if result["tag"].lower() == "incorrect_pose":  # "incorrect_pose"가 비정상 태그로 설정된 경우
            angle = analyze_pose(test_image_path)
            if angle is not None:
                if angle > 170:
                    print("다리가 너무 펴져 있습니다. 약간 굽혀보세요.")
                elif angle < 100:
                    print("다리가 너무 굽혀져 있습니다. 약간 펴보세요.")
                else:
                    print("다리의 각도가 적절합니다.")
        else:
            print("자세가 정상으로 판별되었습니다.")
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"오류 발생: {e}")


Predicted Tag: incorrect_pose, Probability: 0.86
다리의 각도가 적절합니다.


In [10]:
import cv2
import mediapipe as mp
import math

# MediaPipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def extract_skeleton_coordinates(image_path):
    """
    MediaPipe를 사용해 이미지에서 스켈레톤 좌표를 추출합니다.
    :param image_path: 이미지 파일 경로
    :return: 랜드마크 좌표 리스트
    """
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found.")
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            coordinates = [(lm.x, lm.y) for lm in landmarks]
            return coordinates
        else:
            print(f"No pose detected in {image_path}")
            return None

def calculate_coordinate_difference(coords1, coords2):
    """
    두 스켈레톤 좌표 리스트의 차이를 계산합니다.
    :param coords1: 첫 번째 이미지의 좌표 리스트
    :param coords2: 두 번째 이미지의 좌표 리스트
    :return: 각 관절 간 차이 리스트
    """
    if not coords1 or not coords2 or len(coords1) != len(coords2):
        raise ValueError("Coordinates are not valid or do not match in length.")
    
    differences = []
    for coord1, coord2 in zip(coords1, coords2):
        diff = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
        differences.append(diff)
    return differences

def visualize_skeleton(image_path, title):
    """
    MediaPipe를 사용해 이미지에 스켈레톤을 시각화합니다.
    :param image_path: 이미지 파일 경로
    :param title: 창 제목
    """
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found.")
    
    annotated_image = image.copy()
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow(title, annotated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # 정상 자세와 비정상 자세 이미지 경로
    normal_image_path = "473-1-2-21-Z62_A-0000001.jpg"  # 정상 이미지 경로
    incorrect_image_path = "473-1-2-21-Z62_A-0000027.jpg"  # 비정상 이미지 경로
    
    try:
        # 정상 이미지의 스켈레톤 좌표 추출
        normal_coords = extract_skeleton_coordinates(normal_image_path)
        
        # 비정상 이미지의 스켈레톤 좌표 추출
        incorrect_coords = extract_skeleton_coordinates(incorrect_image_path)
        
        if normal_coords and incorrect_coords:
            # 비정상 자세와 정상 자세 비교
            differences = calculate_coordinate_difference(normal_coords, incorrect_coords)
            
            # 결과 출력
            print("스켈레톤 좌표 간 차이:")
            for i, diff in enumerate(differences):
                print(f"랜드마크 {i}: 차이 = {diff:.4f}")
            
            # 스켈레톤 시각화
            print("정상 자세:")
            visualize_skeleton(normal_image_path, "정상 자세 (Normal Pose)")
            print("비정상 자세:")
            visualize_skeleton(incorrect_image_path, "비정상 자세 (Incorrect Pose)")
        else:
            print("스켈레톤 좌표를 추출하지 못했습니다.")
    
    except Exception as e:
        print(f"오류 발생: {e}")


스켈레톤 좌표 간 차이:
랜드마크 0: 차이 = 0.0034
랜드마크 1: 차이 = 0.0033
랜드마크 2: 차이 = 0.0031
랜드마크 3: 차이 = 0.0030
랜드마크 4: 차이 = 0.0035
랜드마크 5: 차이 = 0.0033
랜드마크 6: 차이 = 0.0031
랜드마크 7: 차이 = 0.0017
랜드마크 8: 차이 = 0.0024
랜드마크 9: 차이 = 0.0038
랜드마크 10: 차이 = 0.0033
랜드마크 11: 차이 = 0.0040
랜드마크 12: 차이 = 0.0034
랜드마크 13: 차이 = 0.0051
랜드마크 14: 차이 = 0.0056
랜드마크 15: 차이 = 0.0041
랜드마크 16: 차이 = 0.0108
랜드마크 17: 차이 = 0.0039
랜드마크 18: 차이 = 0.0074
랜드마크 19: 차이 = 0.0051
랜드마크 20: 차이 = 0.0066
랜드마크 21: 차이 = 0.0025
랜드마크 22: 차이 = 0.0079
랜드마크 23: 차이 = 0.0073
랜드마크 24: 차이 = 0.0194
랜드마크 25: 차이 = 0.0350
랜드마크 26: 차이 = 0.0377
랜드마크 27: 차이 = 0.0625
랜드마크 28: 차이 = 0.0696
랜드마크 29: 차이 = 0.0772
랜드마크 30: 차이 = 0.0800
랜드마크 31: 차이 = 0.0827
랜드마크 32: 차이 = 0.0859
정상 자세:
비정상 자세:


# 푸쉬업 포함함

In [ ]:
import cv2
import mediapipe as mp
import math
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 모델 설정
project_settings = {
    "레그레이즈": {
        "ENDPOINT": "",
        "PREDICTION_KEY": "",
        "PROJECT_ID": "",
        "PUBLISHED_NAME": "LEG_RAISE",
        "NORMAL_IMAGE": "473-1-2-21-Z62_A-0000001.jpg"
    },
    "푸쉬업": {
        "ENDPOINT": "",
        "PREDICTION_KEY": "",
        "PROJECT_ID": "",
        "PUBLISHED_NAME": "PUSHUP",
        "NORMAL_IMAGE": "610-1-3-28-Z3_A-0000005.jpg"
    }
}

# MediaPipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def classify_image(file_path, pose_type):
    """
    Custom Vision 모델로 이미지를 분류합니다.
    :param file_path: 분석할 이미지 파일 경로
    :param pose_type: 자세 종류 (레그레이즈, 푸쉬업 등)
    :return: 분류 결과 (태그 이름과 확률)
    """
    if pose_type not in project_settings:
        raise ValueError(f"Unknown pose type: {pose_type}")
    
    settings = project_settings[pose_type]
    credentials = ApiKeyCredentials(in_headers={"Prediction-key": settings["PREDICTION_KEY"]})
    predictor = CustomVisionPredictionClient(settings["ENDPOINT"], credentials)
    
    with open(file_path, "rb") as image_contents:
        results = predictor.classify_image(settings["PROJECT_ID"], settings["PUBLISHED_NAME"], image_contents.read())
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

def determine_exercise(file_path):
    """
    업로드된 이미지가 레그 레이즈인지 푸쉬업인지 판별합니다.
    :param file_path: 분석할 이미지 파일 경로
    :return: 운동 종류 (레그레이즈, 푸쉬업 등)
    """
    for pose_type in project_settings.keys():
        result = classify_image(file_path, pose_type)
        if result["probability"] > 0.8:  # 높은 확률로 운동을 판별
            return pose_type, result
    return None, {"tag": "Unknown", "probability": 0.0}

def calculate_angle(a, b, c):
    """
    세 점의 좌표를 사용해 각도를 계산합니다.
    :param a, b, c: 좌표 (x, y)
    :return: 각도 (degree)
    """
    ab = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    magnitude_ab = math.sqrt(ab[0]**2 + ab[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    if magnitude_ab * magnitude_bc == 0:
        return 0
    cos_theta = dot_product / (magnitude_ab * magnitude_bc)
    angle = math.degrees(math.acos(cos_theta))
    return angle

def analyze_pose(image_path, pose_type):
    """
    이미지에서 자세를 분석하고 각도를 계산합니다.
    :param image_path: 분석할 이미지 파일 경로
    :param pose_type: 자세 종류 (레그레이즈, 푸쉬업 등)
    :return: 계산된 각도 정보
    """
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found.")
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(image_rgb)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            if pose_type == "레그레이즈":
                hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                angle = calculate_angle(hip, knee, ankle)
                return angle
            
            elif pose_type == "푸쉬업":
                shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                angle = calculate_angle(shoulder, hip, ankle)
                return angle
            
        else:
            print("포즈를 감지하지 못했습니다.")
            return None

if __name__ == "__main__":
    uploaded_image_path = "473-1-2-21-Z62_A-0000001.jpg"  # 사용자 업로드 이미지 ##########################################################
    
    try:
        # 운동 종류 판별
        pose_type, result = determine_exercise(uploaded_image_path)
        print(f"운동 종류: {pose_type}")
        print(f"Predicted Tag: {result['tag']}, Probability: {result['probability']:.2f}")
        
        if result["tag"].lower() == "incorrect_pose":
            angle = analyze_pose(uploaded_image_path, pose_type)
            if angle is not None:
                if pose_type == "레그레이즈":
                    if angle > 170:
                        print("다리가 너무 펴져 있습니다. 약간 굽혀보세요.")
                    elif angle < 100:
                        print("다리가 너무 굽혀져 있습니다. 약간 펴보세요.")
                    else:
                        print("다리의 각도가 적절합니다.")
                elif pose_type == "푸쉬업":
                    if angle > 160:
                        print("상체가 너무 기울어져 있습니다. 자세를 조정하세요.")
                    elif angle < 120:
                        print("상체가 너무 낮습니다. 자세를 조정하세요.")
                    else:
                        print("상체의 각도가 적절합니다.")
        else:
            print("자세가 정상으로 판별되었습니다.")
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"오류 발생: {e}")


운동 종류: None
Predicted Tag: Unknown, Probability: 0.00
자세가 정상으로 판별되었습니다.


그냥 푸쉬업, 레그레이즈 정상 비정상인지만 판별하는걸로 출력

In [6]:
import cv2
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# Custom Vision 모델 설정
project_settings = {
    "레그레이즈": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "4799206e-b5be-41a2-9f1e-507aa06e9a89",
        "PUBLISHED_NAME": "LEG_RAISE"
    },
    "푸쉬업": {
        "ENDPOINT": "https://5b001team1customvision-prediction.cognitiveservices.azure.com",
        "PREDICTION_KEY": "2QpZbyO6aXNggNRFOFDalFoMZ0wlH04rIq3vu84rUAVfYLiYtkXBJQQJ99BAACYeBjFXJ3w3AAAIACOGIbb2",
        "PROJECT_ID": "7a47753c-c11b-44af-8ca1-6dbace177a71",
        "PUBLISHED_NAME": "PUSHUP"
    }
}

def classify_image(file_path, pose_type):
    """
    Custom Vision 모델로 이미지를 분류합니다.
    :param file_path: 분석할 이미지 파일 경로
    :param pose_type: 자세 종류 (레그레이즈, 푸쉬업 등)
    :return: 분류 결과 (태그 이름과 확률)
    """
    settings = project_settings[pose_type]
    credentials = ApiKeyCredentials(in_headers={"Prediction-key": settings["PREDICTION_KEY"]})
    predictor = CustomVisionPredictionClient(settings["ENDPOINT"], credentials)

    with open(file_path, "rb") as image_contents:
        results = predictor.classify_image(settings["PROJECT_ID"], settings["PUBLISHED_NAME"], image_contents.read())
    top_prediction = results.predictions[0]
    return {"tag": top_prediction.tag_name, "probability": top_prediction.probability}

def analyze_pose(file_path, pose_type):
    """
    입력받은 운동 종류(레그레이즈/푸쉬업)에 따라 자세를 분석합니다.
    :param file_path: 분석할 이미지 파일 경로
    :param pose_type: 운동 종류
    :return: 정상/비정상 자세 판별 결과
    """
    if pose_type not in project_settings:
        raise ValueError(f"지원하지 않는 운동 종류입니다: {pose_type}")

    result = classify_image(file_path, pose_type)
    if result["tag"].lower() == "incorrect_pose":
        return "비정상 자세로 판별되었습니다."
    else:
        return "정상 자세로 판별되었습니다."

if __name__ == "__main__":
    # 외부에서 운동 종류를 입력받음 (예: 사용자 입력, API 요청 등)
    pose_type = input("운동 종류를 입력하세요 (레그레이즈/푸쉬업): ").strip()
    uploaded_image_path = "473-1-2-21-Z62_A-0000027.jpg"  # 사용자 업로드 이미지

    try:
        # 자세 분석
        result = analyze_pose(uploaded_image_path, pose_type)
        print(f"운동 종류: {pose_type}")
        print(result)

    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"오류 발생: {e}")


운동 종류: 레그레이즈
비정상 자세로 판별되었습니다.
